In [1]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing as preproc
from scipy import stats
import numpy as np

In [2]:
df=pd.read_parquet("diabetic_data_cleaned_pca_som_diag_grouped_ae.parquet")

In [3]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,ae_4,ae_5,ae_6,ae_7,ae_8,ae_9,ae_10,ae_11,ae_12,ae_13
0,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,...,8.584562,20.436142,5.291698,8.157053,17.783960,23.949799,11.668662,8.264934,22.667690,27.363850
1,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,...,15.110230,15.245245,33.812019,2.966186,29.897364,28.222050,23.203943,12.367907,29.435223,36.155518
2,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,...,1.598465,25.428694,8.711449,21.261776,11.921114,30.430517,22.689114,7.689127,13.135617,25.826536
3,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,13.074265,8.245643,12.659610,19.848545,6.032684,29.801752,10.818433,3.274082,20.075939,10.891075
4,35754,82637451,Caucasian,Male,[50-60),2,1,2,3,31,...,31.112360,17.769810,11.906863,35.779766,28.849283,11.464336,28.415997,17.790482,19.180479,26.632393


In [4]:
df.columns, df.shape

(Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
        'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
        'time_in_hospital', 'num_lab_procedures', 'num_procedures',
        'num_medications', 'number_outpatient', 'number_emergency',
        'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
        'metformin', 'repaglinide', 'glimepiride', 'glipizide', 'glyburide',
        'pioglitazone', 'rosiglitazone', 'insulin', 'glyburide-metformin',
        'change', 'diabetesMed', 'readmitted', 'PC1', 'PC2', 'PC3',
        'som_jaccard_0', 'som_jaccard_1', 'som_euc_0', 'som_euc_1',
        'diag_1_group', 'diag_2_group', 'diag_3_group', 'ae_0', 'ae_1', 'ae_2',
        'ae_3', 'ae_4', 'ae_5', 'ae_6', 'ae_7', 'ae_8', 'ae_9', 'ae_10',
        'ae_11', 'ae_12', 'ae_13'],
       dtype='object'),
 (100244, 55))

In [5]:
DB_COLUMNS = [
  'PC1','PC2', 'PC3',
  'ae_0', 'ae_1', 'ae_2', 'ae_3', 'ae_4',
  'ae_5', 'ae_6', 'ae_7', 'ae_8', 'ae_9',
  'ae_10', 'ae_11', 'ae_12', 'ae_13'
]

df[DB_COLUMNS]

,PC1,PC2,PC3,ae_0,ae_1,ae_2,ae_3,ae_4,ae_5,ae_6,ae_7,ae_8,ae_9,ae_10,ae_11,ae_12,ae_13
0,-0.214843,0.070744,-0.194516,21.509342,19.395063,13.789462,18.633125,8.584562,20.436142,5.291698,8.157053,17.783960,23.949799,11.668662,8.264934,22.667690,27.363850
1,0.417941,-0.522845,0.160733,35.920300,9.260715,18.687860,9.814375,15.110230,15.245245,33.812019,2.966186,29.897364,28.222050,23.203943,12.367907,29.435223,36.155518
2,-0.129162,-0.131116,-0.050765,25.516636,20.602079,21.286129,11.283273,1.598465,25.428694,8.711449,21.261776,11.921114,30.430517,22.689114,7.689127,13.135617,25.826536
3,-0.333245,-0.143083,-0.009183,15.020832,18.029791,14.233169,6.268053,13.074265,8.245643,12.659610,19.848545,6.032684,29.801752,10.818433,3.274082,20.075939,10.891075
4,0.640729,-0.446888,-0.059434,17.637230,34.109043,10.788097,20.966002,31.112360,17.769810,11.906863,35.779766,28.849283,11.464336,28.415997,17.790482,19.180479,26.632393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100239,-0.226515,0.049629,-0.151316,24.948250,13.505182,24.837986,13.648841,3.809228,23.910847,27.008827,18.464516,17.114193,40.147312,16.920292,33.196503,9.398462,8.666927
100240,0.266458,-0.086568,-0.008378,15.829528,23.370535,19.452053,12.653931,11.257184,11.205988,19.306614,6.731514,17.339216,16.855181,6.723785,12.657176,4.543801,27.678026
100241,-0.277035,-0.027778,-0.406056,20.020979,7.852528,25.541323,14.455379,3.585193,16.499947,16.587612,21.113573,18.614311,36.455925,17.439417,15.958732,14.460878,5.931040
100242,0.291241,0.333055,0.077571,3.588265,12.805207,15.814934,7.906501,7.093519,18.012829,12.256166,0.000000,5.988210,12.247410,1.099811,15.101754,17.273737,13.195094


In [6]:
X=df[DB_COLUMNS]

# Scale the independent variables
#scaler = StandardScaler()
X_scaled = preproc.MinMaxScaler().fit_transform(X)

In [7]:
# Apply DBSCAN
epsilon = 0.40 # You may need to adjust this parameter
min_samples = 10 # You may need to adjust this parameter
dbscan = DBSCAN(eps=epsilon, min_samples=min_samples, n_jobs=-1)
#dbscan.fit(X_scaled)
dbscan.fit(X_scaled)

DBSCAN(eps=0.4, min_samples=10, n_jobs=-1)

In [8]:
# Extract outlier labels
outlier_mask = dbscan.labels_ == -1

# Get the outliers
outliers = X[outlier_mask]

print(len(outliers))

1178


In [9]:
len(set(dbscan.labels_))

2

In [10]:
# Analyze outliers
print(len(df[outlier_mask]))

1178


In [11]:
non_outlier_mask=dbscan.labels_ != -1

In [12]:
print(len(df[non_outlier_mask]))

99066


In [13]:
def display_df(df: pd.DataFrame, title: str = ""):
  display(df.style.hide(axis="index").set_caption(title))

In [14]:
TGT = "readmitted"

In [15]:
prop_total = df.loc[:, TGT].value_counts().reset_index()
display_df(prop_total.sort_values("readmitted", ascending = False), "full dataset")

prop_outs = df.loc[list(outlier_mask), TGT].value_counts().reset_index()
display_df(prop_outs.sort_values("readmitted", ascending = False), "DBscan outliers")

stats.chi2_contingency(np.array([prop_total["count"].values, prop_outs["count"].values]))

readmitted,count
NO,53821
>30,35173
<30,11250


readmitted,count
NO,609
>30,429
<30,140


Chi2ContingencyResult(statistic=1.9022834936962831, pvalue=0.3862997150890829, dof=2, expected_freq=array([[53797.80442113, 35188.4885725 , 11257.70700637],
       [  632.19557887,   413.5114275 ,   132.29299363]]))

In [16]:
output_df=df[non_outlier_mask]

In [17]:
filename = "diabetic_data_cleaned_pca_som_diag_grouped_ae_dbscan"
output_df.to_csv(f"{filename}.csv")
output_df.to_parquet(f"{filename}.parquet")